![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
import pandas as pd
import numpy as np
qb = QuantBook()
import statsmodels as sm
from statsmodels.tsa.stattools import coint, adfuller
spy = qb.AddEquity("SPY")
apple = qb.AddEquity("AAPL")

history = qb.History(qb.Securities.Keys, 50, Resolution.Daily)
history

In [15]:
tickers = [ 'XLK', 'QQQ', 'BANC', 'BBVA', 'BBD', 'BCH', 'BLX', 'BSBR', 'BSAC', 'SAN',
            'CIB', 'BXS', 'BAC', 'BOH', 'BMO', 'BK', 'BNS', 'BKU', 'BBT','NBHC', 'OFG']#,
            #'BFR', 'CM', 'COF', 'C', 'VLY', 'WFC', 'WAL', 'WBK','RBS', 'SHG', 'STT', 'STL', 'SCNB', 'SMFG', 'STI']
            # 'DKT', 'DB', 'EVER', 'KB', 'KEY', , 'MTB', 'BMA', 'MFCB', 'MSL', 'MTU', 'MFG', 
            # 'PVTD', 'PB', 'PFS', 'RF', 'RY', 'RBS', 'SHG', 'STT', 'STL', 'SCNB', 'SMFG', 'STI',
            # 'SNV', 'TCB', 'TD', 'USB', 'UBS', 'VLY', 'WFC', 'WAL', 'WBK', 'WF', 'YDKN', 'ZBK']

symbols = pd.Series
for i in tickers:
    symbols.concat(Equity(str(i)))
#sym = pd.DataFrame(data = symbols)
hist = qb.History(str(symbols), 50, Resolution.Daily)
hist


In [32]:
returns = history.unstack(level = 1).close.transpose().pct_change().dropna()
returns# = pd.DataFrame(returns)
def adf2(returns, siglevel=0.05):
    return pd.Series([adfuller(values)[1] < siglevel for columns, values in returns.iteritems()],\
        index = returns.columns)

stat = adf2(returns)
stat


In [33]:
stattested = []
for sym, val in stat.iteritems():
    if val == True:
        stattested.append(sym)
stattested

In [34]:

stathist = qb.History(stattested, 50, Resolution.Daily)
corr = stathist.unstack(level=1).close.transpose().corr(method='kendall').dropna()
corr

In [41]:
corr1 = corr.unstack().drop_duplicates()
corr1 = pd.DataFrame(corr1)
corr1
CoarseSelection = corr1.rename({0:'correlation'}, axis=1)
CoarseSelection

In [42]:
for correlation, row in CoarseSelection.iterrows():
    if row['correlation'] == 1 or row['correlation'] < 0.7:
        CoarseSelection.drop(CoarseSelection[(CoarseSelection['correlation'] == 1)| \
            (CoarseSelection['correlation'] < 0.7)].index, inplace=True) 
CoarseSelection

In [43]:
syms = CoarseSelection.drop([('correlation')],axis=1)
pairs = list(CoarseSelection.index)

In [44]:
pairs

In [46]:
from statsmodels.tsa.stattools import coint

def cointtest(pairs, siglevel=0.05):
    for i, j in pairs:
        historyi = qb.History(qb.Symbol(str(i)), 50, Resolution.Daily)
        historyj = qb.History(qb.Symbol(str(j)), 50, Resolution.Daily)
        returnsi = historyi.unstack(level = 1).close.transpose().pct_change().dropna()
        returnsj = historyj.unstack(level = 1).close.transpose().pct_change().dropna()
        
        return pd.Series([coint(returnsi, returnsj)[1] < siglevel for columns, values in \
                         pairs], index = pairs)

In [56]:
copairs = cointtest(pairs)
cotested = []
for sym, val in copairs.iteritems():
    if val == True:
        cotested.append(sym)
cop = cotested

In [60]:
cop = pd.DataFrame(cop)

In [76]:
qb.AddUniverse((for cf in coarse:
                   sec = s.GetOrADD(cf.Symbol, sym = SelectionData())
                   sec.Update(cf.EndTime, cf.Price)
                   sec.ScaledDelta
                   cf.Symbol).Take(Count))

In [78]:
self.AddUniverse(coarse = for cf in coarse)